# Download and virtualize MUR-SST data

In [ ]:
from pathlib import Path

import earthaccess
from virtualizarr import open_virtual_dataset

### Authenticate via earthaccess

In [ ]:
earthaccess.login()

### Download MUR-SST dataset

In [ ]:
results = earthaccess.search_data(
    concept_id="C1996881146-POCLOUD", count=1, temporal=("2002-06-01", "2002-06-01")
)
fp = earthaccess.download(results, "earthaccess_data")[0]

### Virtualize MUR-SST dataset

In [ ]:
def virtualize_dataset(local_fp):
    """Create a virtual reference file for a dataset"""

    def local_to_s3_url(old_local_path: str) -> str:
        """Replace local path to s3 uri for all chucks"""

        new_s3_bucket_url = Path("/".join(s3_uri.split("/")[:-1]))

        filename = Path(old_local_path).name
        return str(new_s3_bucket_url / filename)

    s3_uri = results[0].data_links(access="direct")[0]
    output_fp = f"earthaccess_data/{s3_uri.split('/')[-1].strip('.nc')}.json"
    virtual_ds = open_virtual_dataset(str(local_fp), indexes={})
    virtual_ds.virtualize.rename_paths(local_to_s3_url)
    virtual_ds.virtualize.to_kerchunk(output_fp, format="json")

In [ ]:
virtualize_dataset(fp)